In [1]:
# Import packages
import time
import pyspark
import os
import csv
from pyspark.mllib.regression import LabeledPoint
from pyspark import SparkContext, SparkConf

In [2]:
#Start a spark session
sc =SparkContext()

Exception: Java gateway process exited before sending the driver its port number

In [ ]:
train_path='train.csv'
test_path='test.csv'
# Load csv file as RDD
train_rdd = sc.textFile(train_path)
test_rdd = sc.textFile(test_path)

In [3]:
train_rdd.take(3)

NameError: name 'train_rdd' is not defined

In [4]:
    # Parse RDD to DF
def parseTrain(rdd):
 
    # extract data header (first row)
    header = rdd.first()
    # remove header
    body = rdd.filter(lambda r: r!=header)
def parseRow(row):
        # a function to parse each text row into
        # data format
        # remove double quote, split the text row by comma
        row_list = row.replace('&quot;','').split(&quot;,&quot;)
        # convert python list to tuple, which is
        # compatible with pyspark data structure
        row_tuple = tuple(row_list)
        return row_tuple
 
    rdd_parsed = body.map(parseRow)
 
    colnames = header.split(&quot;,&quot;)
    colnames.insert(3,'FirstName')
 
    return rdd_parsed.toDF(colnames)
 
def parseTest(rdd):
    header = rdd.first()
    body = rdd.filter(lambda r: r!=header)
 
    def parseRow(row):
        row_list = row.replace('&quot;','').split(&quot;,&quot;)
        row_tuple = tuple(row_list)
        return row_tuple
 
    rdd_parsed = body.map(parseRow)
 
    colnames = header.split(&quot;,&quot;)
    colnames.insert(2,'FirstName')
 
    return rdd_parsed.toDF(colnames)
 
train_df = parseTrain(train_rdd)
test_df = parseTest(test_rdd)

IndentationError: expected an indented block (<ipython-input-4-2dfb2bf30bc2>, line 5)

In [5]:
train_df.show(3)

NameError: name 'train_df' is not defined

In [6]:
#Combine Train/Test Data

## Add Survived column to test
from pyspark.sql.functions import lit, col
train_df = train_df.withColumn('Mark',lit('train'))
test_df = (test_df.withColumn('Survived',lit(0))
                  .withColumn('Mark',lit('test')))
test_df = test_df[train_df.columns]
## Append Test data to Train data
df = train_df.unionAll(test_df)

NameError: name 'train_df' is not defined

In [7]:
#Convert Age, SibSp, Parch, Fare to Numeric

df = (df.withColumn('Age',df['Age'].cast(&quot;double&quot;))
            .withColumn('SibSp',df['SibSp'].cast(&quot;double&quot;))
            .withColumn('Parch',df['Parch'].cast(&quot;double&quot;))
            .withColumn('Fare',df['Fare'].cast(&quot;double&quot;))
            .withColumn('Survived',df['Survived'].cast(&quot;double&quot;))
            )
df.printSchema()

SyntaxError: invalid syntax (<ipython-input-7-f46e1b5f24e4>, line 3)

In [8]:
#Impute missing Age and Fare with the Average
numVars = ['Survived','Age','SibSp','Parch','Fare']
def countNull(df,var):
    return df.where(df[var].isNull()).count()
 
missing = {var: countNull(df,var) for var in numVars}
age_mean = df.groupBy().mean('Age').first()[0]
fare_mean = df.groupBy().mean('Fare').first()[0]
df = df.na.fill({'Age':age_mean,'Fare':fare_mean})

NameError: name 'df' is not defined

In [9]:
# Feature Engineering
# Extract Title from Name

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
 
## created user defined function to extract title
getTitle = udf(lambda name: name.split('.')[0].strip(),StringType())
df = df.withColumn('Title', getTitle(df['Name']))
 
df.select('Name','Title').show(3)

NameError: name 'df' is not defined

In [10]:
# Index categorical variable
catVars = ['Pclass','Sex','Embarked','Title']
 
## index Sex variable
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol = 'Sex', outputCol = 'Sex_indexed')
df_indexed = si.fit(df).transform(df).drop('Sex').withColumnRenamed('Sex_indexed','Sex')
 
## make use of pipeline to index all categorical variables
def indexer(df,col):
    si = StringIndexer(inputCol = col, outputCol = col+'_indexed').fit(df)
    return si
 
indexers = [indexer(df,col) for col in catVars]
 
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = indexers)
df_indexed = pipeline.fit(df).transform(df)
 
df_indexed.select('Embarked','Embarked_indexed').show(3)

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

In [11]:
# Convert to label/features format
catVarsIndexed = [i+'_indexed' for i in catVars]
featuresCol = numVars+catVarsIndexed
featuresCol.remove('Survived')
labelCol = ['Mark','Survived']
 
from pyspark.sql import Row
from pyspark.mllib.linalg import DenseVector
row = Row('mark','label','features')
 
df_indexed = df_indexed[labelCol+featuresCol]
# 0-mark, 1-label, 2-features
# map features to DenseVector
lf = df_indexed.map(lambda r: (row(r[0], r[1],DenseVector(r[2:])))).toDF()
# index label
# convert numeric label to categorical, which is required by
# decisionTree and randomForest
lf = StringIndexer(inputCol = 'label',outputCol='index').fit(lf).transform(lf)
 
lf.show(3)

NameError: name 'df_indexed' is not defined

In [12]:
# split back train/test data
train = lf.where(lf.mark =='train')
test = lf.where(lf.mark =='test')
 
# random split further to get train/validate
train,validate = train.randomSplit([0.7,0.3],seed =121)
 
print 'Train Data Number of Row: '+ str(train.count())
print 'Validate Data Number of Row: '+ str(validate.count())
print 'Test Data Number of Row: '+ str(test.count())

SyntaxError: invalid syntax (<ipython-input-12-3adaf1fcf9b7>, line 8)

In [13]:
# Logistic Regression
from pyspark.ml.classification import LogisticRegression
 
# regPara: lasso regularisation parameter (L1)
lr = LogisticRegression(maxIter = 100, regParam = 0.05, labelCol='index').fit(train)
 
# Evaluate model based on auc ROC(default for binary classification)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
 
def testModel(model, validate = validate):
    pred = model.transform(validate)
    evaluator = BinaryClassificationEvaluator(labelCol = 'index')
    return evaluator.evaluate(prod)
 
print 'AUC ROC of Logistic Regression model is: '+str(testModel(lr))

SyntaxError: invalid syntax (<ipython-input-13-68c91a9db0f1>, line 15)

In [14]:
# Decision Tree and Random Forest
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
 
dt = DecisionTreeClassifier(maxDepth = 3, labelCol ='index').fit(train)
rf = RandomForestClassifier(numTrees = 100, labelCol = 'index').fit(train)
 
models = {'LogisticRegression':lr,
          'DecistionTree':dt,
          'RandomForest':rf}
 
modelPerf = {k:testModel(v) for k,v in models.iteritems()}
 
print modelPerf

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-14-2c787e343905>, line 13)